In [1]:
# Change these variables to the intended area
location_name = 'hagerstown' #'hagerstown'

#get zipcodes of intended state/s
from uszipcode import SearchEngine
search = SearchEngine()
data = search.by_state("Maryland",returns=0)
data2 = search.by_state("Pennsylvania", returns=0)

zip_codes1 = [i.zipcode for i in data]
zip_codes1 = [int(zipcode) for zipcode in zip_codes1]

zip_codes2 = [i.zipcode for i in data2]
zip_codes2 = [int(zipcode) for zipcode in zip_codes2]

zip_codes = zip_codes1+zip_codes2

#print(zip_codes)

#set cbg to cluster on and the min population
core_cbg = '240430006012' #401139400082
min_cluster_pop = 10000


/opt/homebrew/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
import geopandas as gpd
import numpy as np

import pandas as pd

import networkx as nx
import json
import pickle

In [3]:
#Read safegraph files
filename = location_name + '.csv'

try:
    
    df = pd.read_csv(filename)
except:
    datalist = []

    with pd.read_csv('patterns.csv', chunksize=10000) as reader:
        for chunk in reader:
            datalist.append(chunk[chunk['postal_code'].isin(zip_codes)])

    df = pd.concat(datalist, axis=0)
    del datalist

    try:
        
        df['poi_cbg'] = df['poi_cbg'].astype('int64')
    except:
        print(df['poi_cbg'])
        pass
    df.to_csv(filename)

In [4]:
cols = ['location_name', 'safegraph_place_id', 'latitude', 'longitude']
filename = location_name + '.pois.csv'


for _, row in df.iterrows():
    if not row['visitor_daytime_cbgs']:
        continue
    
    for visitor_cbg in json.loads(row['visitor_daytime_cbgs']).keys():
        
        if visitor_cbg not in zip_codes:
            zip_codes.append(visitor_cbg)

try:
    poif = pd.read_csv(filename)
except:
    datalist = []

    with pd.read_csv('2021_05_05_03_core_poi.csv', chunksize=10000) as reader:
        for chunk in reader:
            datalist.append(chunk[chunk['postal_code'].isin(zip_codes)])

    poif = pd.concat(datalist, axis=0)
    del datalist

    poif.to_csv(filename)


In [5]:
#Try to avoid this function especially in loops since it takes time
def cbg_geocode(cbg):
    '''Generates the latitude and longitude coordinates for a CBG

    Args:
        cbg (str): CBG ID
        
    Returns:
        pd.Series: ID, Latitude, & Longitude coords for a CBG
    '''
    lat = []
    long = []

   
    for _, poi in df.loc[df['poi_cbg'] == int(cbg)].iterrows():
        poi_info = poif.loc[poif['safegraph_place_id'] == poi['safegraph_place_id']]
        lat.append(poi_info.iloc[0]['latitude'])
        long.append(poi_info.iloc[0]['longitude'])
            
    return pd.Series(data={
        'label': str(cbg),
        'latitude': np.mean(lat),
        'longitude': np.mean(long)
    })

In [6]:
def poi_geocode(poi):
    '''Finds the latitude and longitude coordinates for a POI

    Args:
        poi (pd.Series): Series that contains the SafeGraph data for a specific POI

    Returns:
        pd.Series: Name, Latitude, & Longitude coords for a POI
    '''
    place_id = poi if type(poi) == str else poi['safegraph_place_id']
    poi_data = poif.loc[poif['safegraph_place_id'] == place_id]
    return pd.Series(data={
        'label': place_id,
        'latitude': poi_data['latitude'],
        'longitude': poi_data['longitude']
    })

In [7]:
#function to find distances between cbgs, used in the distance graph
from math import sin, cos, atan2, pi, sqrt
def distance(lat1, long1, lat2, long2):
    lat1 = lat1*pi/180
    long1 = long1*pi/180
    lat2 = lat2*pi/180
    long2 = long2*pi/180

    Radius = 6371
    haversine = sin((lat2-lat1)/2)**2 + cos(lat1)*cos(lat2) * sin((long2-long1)/2)**2
    c = 2*atan2(sqrt(haversine), sqrt(1-haversine))
    dist = Radius*c
    return dist



In [8]:
#CBG shapefiles, to find them go to https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html , find the year you want and under Download choose
# "FTP Archive by State"


#Make sure shapefiles agree with safegraph by creating the map with POI's included

#2022 Shapefiles
# gdf1 = gpd.read_file('tl_rd22_24_bg/tl_rd22_24_bg.shp')
# gdf2 = gpd.read_file('tl_rd22_42_bg/tl_rd22_42_bg.shp')
# gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))


#2010 Shapefiles use these for safegraph (safegraph data is from 2021 but they use 2010 cbgs)
gdf1 = gpd.read_file('tl_2010_24_bg10/tl_2010_24_bg10.shp')
gdf2 = gpd.read_file('tl_2010_42_bg10/tl_2010_42_bg10.shp')
gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))





In [9]:
def gen_graph(df, weighted=True):
    '''Reads from a SafeGraph dataframe of a certain location

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location
        weighted (bool, optional): Whether the result should have
            weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs and the edges and weights
            are determined by the visitors to and from a node to other nodes
    '''
    G = nx.Graph()
    regg_count = 0
    for _, row in df.iterrows():
        poi_cbg = str(row['poi_cbg'])
        
        try:

            int_poi = int(float(poi_cbg))
            poi_cbg = str(int_poi)
        except:
            regg_count+=1
            continue
        
        G.add_node(poi_cbg, pois = [])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        weight = row['median_dwell'] if weighted else 1
        
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            #print(poi_cbg, visitor_cbg, num_visitors, weight)
            
            
            if visitor_cbg == poi_cbg:
                continue # Ignore edges that connect and come from the same node
            
            # visitor_latitiude = cbg_geocode(visitor_cbg)[1]
            # visitor_long = cbg_geocode(visitor_cbg)[2]
            
            # poi_latitude = cbg_geocode(poi_cbg)[1]
            # poi_long = cbg_geocode(poi_cbg)[2]

            
            #dist = distance(visitor_latitiude,visitor_long,poi_latitude,poi_long)
            #print(visitor_latitiude,  ' ', visitor_long, ' ', poi_latitude, ' ', poi_long)
            # if (np.isnan(visitor_latitiude) or np.isnan(visitor_long) or np.isnan(poi_latitude) or np.isnan(poi_long)):
            #     continue

            
            
            
            if G.has_edge(visitor_cbg, poi_cbg):
                try:
                    G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
                except ZeroDivisionError:
                    continue
            else:
                try:
                    G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
                except ZeroDivisionError:
                    continue
            # try:
            #     G[visitor_cbg][poi_cbg] += int(num_visitors)
            # except:
            #     G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors))])
        
        # Remove nodes without any edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)

    UG = G.to_undirected()
    for node in G:
        for node2 in nx.neighbors(G, node):
            if node in nx.neighbors(G, node2):
                UG.edges[node, node2]['weight'] = G.edges[node, node2]['weight'] + G.edges[node2, node]['weight'] 
    
    print('G has %d nodes and %d edges.' % (nx.number_of_nodes(UG), nx.number_of_edges(UG)))
    print("bad data ", regg_count)
    return UG

In [10]:
#distance graph where the weights are divided by the distance of the POI from the seed
def gen_d_graph(df, gdf, weighted=True):
    '''Reads from a SafeGraph dataframe of a certain location

    Args:
        df (pd.DataFrame): The SafeGraph dataset of a particular location
        weighted (bool, optional): Whether the result should have
            weighted edges. Defaults to True.

    Returns:
        nx.Graph: Graph where nodes are the CBGs and the edges and weights
            are determined by the visitors to and from a node to other nodes
    '''
    G = nx.Graph()
    seed_row = gdf[gdf['GEOID'] == '240430006012']
    seed_latitiude = seed_row['latitude'].iloc[0]
    seed_long = seed_row['longitude'].iloc[0]
    regg_count = 0
    for _, row in df.iterrows():
        poi_cbg = str(row['poi_cbg'])
        
        try:

            int_poi = int(float(poi_cbg))
            poi_cbg = str(int_poi)
        except:
            regg_count+=1
            continue
        
        G.add_node(poi_cbg, pois = [])
        G.nodes[poi_cbg]['pois'].append(row['safegraph_place_id'])
        
        weight = row['median_dwell'] if weighted else 1
        
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            #print(poi_cbg, visitor_cbg, num_visitors, weight)
            
            
            if visitor_cbg == poi_cbg:
                continue # Ignore edges that connect and come from the same node
            
           
            try:
                
                
                poi_row = gdf[gdf['GEOID'] == poi_cbg]
                poi_latitude = poi_row['latitude'].iloc[0]
                poi_long = poi_row['longitude'].iloc[0]
               
            except IndexError:
                regg_count +=1
                continue

            
            
            dist = int(distance(seed_latitiude,seed_long,poi_latitude,poi_long))
            if G.has_edge(visitor_cbg, poi_cbg):
                try:
                    G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)/dist
                except ZeroDivisionError:
                    G[visitor_cbg][poi_cbg]['weight'] += int(num_visitors * weight)
                   
            else:
                try:
                    G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight)/dist)])
                except ZeroDivisionError:
                    G.add_weighted_edges_from([(visitor_cbg, poi_cbg, int(num_visitors * weight))])
                    
           
        
        # Remove nodes without any edges
        if G.degree[poi_cbg] == 0:
            G.remove_node(poi_cbg)

    #Change to undirected graph
    UG = G.to_undirected()
    for node in G:
        for node2 in nx.neighbors(G, node):
            if node in nx.neighbors(G, node2):
                UG.edges[node, node2]['weight'] = G.edges[node, node2]['weight'] + G.edges[node2, node]['weight'] 
    
    print('G has %d nodes and %d edges.' % (nx.number_of_nodes(UG), nx.number_of_edges(UG)))
    print("bad data ", regg_count)
    return UG

In [11]:
#get population of a cbg
cbg_pops = pd.read_csv('data/safegraph_cbg_population_estimate.csv', index_col='census_block_group')


def cbg_population(cbg):
    try:
        return int(cbg_pops.loc[int(cbg)].B00002e1)
    except (ValueError, TypeError):
        return 0

    

In [12]:
#get coordinates
gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
gdf['coords'] = [coords[0] for coords in gdf['coords']]
gdf['longitude'] = gdf['coords'].apply(lambda x: x[0])
gdf['latitude'] = gdf['coords'].apply(lambda x: x[1])
print('creating graph')


#To create regular graph
G = gen_graph(df)



#To create distance graph (save this graph, it takes long to create)

# G = gen_d_graph(df,gdf) for distance graph or just load (below)



# Save graph

# with open("dist_graph.pkl", "wb") as file:
#     pickle.dump(G, file)

#Load graph
# with open("dist_graph.pkl", "rb") as file:
#     G = pickle.load(file)



creating graph
G has 72168 nodes and 1530512 edges.
bad data  1


In [13]:
#To save graph as csv
import pandas as pd
import numpy as np

def graph_to_csv(UG, filename='adj_graph.csv'):
    #Get adjacency matrix
    adj_matrix = nx.adjacency_matrix(UG)

    array_adj_matrix = adj_matrix.toarray()

    # Create the dataframe
    nodes = list(UG.nodes())
    df_adj_matrix = pd.DataFrame(array_adj_matrix, index=nodes, columns=nodes)

    
    df_adj_matrix.to_csv(filename)

    print('Graph written')




In [14]:
#TEST

#poi_geocode('sg:008db39e93a44074bfa1e4fe085fa258')
poi_geocode('sg:008db39e93a44074bfa1e4fe085fa258')[2].iloc[0]
# print(x.iloc[0])



IndexError: single positional indexer is out-of-bounds

In [15]:
#TEST
cbg_geocode('240430005001')

label        240430005001
latitude        39.644759
longitude      -77.716671
dtype: object

In [16]:
#Get total population
total_cbgs_population = 0
for cbg in df.poi_cbg.drop_duplicates().tolist():
   
    #print(cbg)
    total_cbgs_population += cbg_population(cbg)
total_cbgs_population

775923

In [ ]:
#DO NOT USE only left here to show it was tried
#Chooses cbg that will result in the greatest ratio of movement for S, however it's too shortsighted and results in a bad cluster
def greedy_ratio(C, u0, min_pop):
    
    cluster = [ u0 ]
    population = cbg_population(u0)
    it = 1
    while population < min_pop:
        print(population)
        all_adj_cbgs = []
        for i in cluster:
            adj_cbgs = list(C.adj[i])
            
            for j in adj_cbgs:
                if j not in all_adj_cbgs and j not in cluster:
                    all_adj_cbgs.append(j)
        movement_in = 0
        movement_out = 0
        max_ratio = 0
        cbg_to_add = 0
        
       
        
        for i in all_adj_cbgs:
            
            cluster.append(i)
            for j in cluster:
                adj = list(C.adj[j])
                for k in adj:
                    if k in cluster:
                        movement_in += C.adj[j][k]['weight']
                    else:
                        movement_out += C.adj[j][k]['weight']
            ratio = movement_in/(movement_in+movement_out)
            
            if ratio > max_ratio:
                max_ratio = ratio
                
                cbg_to_add = i
            cluster.remove(i)
        
        if cbg_to_add == 0:
            break
        cluster.append(cbg_to_add)
        
        print("Iteration #  " , it, end="", flush=True)
        print("  Prev pop  ", population, end="", flush=True)
        population += cbg_population(cbg_to_add)
        print("  New pop  ", population, end="", flush=True)
        print("  Movement in S  " ,movement_in, end="", flush=True)
        print("  Move out S  ", movement_out, end="", flush=True)
        print("  Ratio  ", ratio)
        print("")
        it+=1
    
    return cluster, population

            
        
        
    
   

        
                
       

In [17]:
def greedy_weight(C, u0, min_pop):
    
    cluster = [ u0 ]
    population = cbg_population(u0)
    it = 1
    while population < min_pop:
        #get all adjacent cbgs of the cluster
        all_adj_cbgs = []
        for i in cluster:
            adj_cbgs = list(C.adj[i])
            
            for j in adj_cbgs:
                if j not in all_adj_cbgs and j not in cluster:
                    all_adj_cbgs.append(j)
        movement_in = 0
        movement_out = 0
        max_movement = 0
        cbg_to_add = 0
        
        
        #Calculate the movement from S to all adjacent cbgs
        for i in all_adj_cbgs:
            current_movement = 0
            
        
            
            for j in cluster:
                try:
                    current_movement += C.adj[i][j]['weight']
                except:
                    pass
            #find the cbg with the greatest movement and add it to S
            if current_movement > max_movement:
                max_movement = current_movement
                cbg_to_add = i
        
        
            
        #Movement checking at each iteration
            
        # for i in cluster:
        #     adj = list(C.adj[i])
        #     for k in adj:
        #         if k in cluster:
        #             movement_in += C.adj[i][k]['weight']/2
        #         else:
        #             movement_out += C.adj[i][k]['weight']
        # ratio = movement_in/(movement_in+movement_out)
        cluster.append(cbg_to_add)
        # print("Iteration #  " , it, end="", flush=True)
        print("  Prev pop  ", population, end="", flush=True)
        population += cbg_population(cbg_to_add)
        print("  New pop  ", population, end="", flush=True)
        # print("  Movement in S  " ,movement_in, end="", flush=True)
        # print("  Move out S  ", movement_out, end="", flush=True)
        # print("  Ratio  ", ratio)
        # print("")
        # it+=1
    
    return cluster, population

            
        
        
    
   

        
                
       

In [ ]:
#Pagerank Nibble function. Uses random walks from the seed to determine 'important' cbgs to add
#https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html
#Alpha value is the chance we restart on seed during a random walk. Ex. alpha = 0.85, there is a 15% chance when walking to a cbg we reset at the node (0.85 is the recommended value)
#Tolerance = 10^-4 means we stop calculating pagerank scores when the change is < 10^-4
def pagerank_nibble(G, u0, alpha=0.85, eps=1.0e-4):
    #Calculate pagerank score of cbgs
    pr = nx.pagerank(G, alpha=alpha, personalization={u0: 1}, tol=1.0e-6)
    pr_items = list(pr.items())
    pr_items.sort(key=lambda x: x[1], reverse=True)

   
    cluster = {u0}
    
    #conductance is a measure of how isolated the cbgs are
    min_conductance = np.inf
    it = 0
    for cbg, p_score in pr_items:
        it +=1
        print(it)
        if p_score < eps:
            break
        cluster.add(cbg)
        conductance = nx.conductance(G, cluster)
        if conductance < min_conductance:
            min_conductance = conductance
            best_cluster = cluster.copy()
            
    return best_cluster, 1





In [18]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import folium
from folium import plugins
from folium import Marker

In [ ]:
print('starting algo')
algorithm_result = greedy_ratio(G, core_cbg, min_cluster_pop)
print('algo done')
print('cluster population:', algorithm_result[1], '# of cbgs:', len(algorithm_result[0]))
print(algorithm_result[0])
movement_in_cbgs = 0
movement_out_cbgs = 0

#This is raw movement, use code below
# for i in algorithm_result[0]:
#     for _, row in df.iterrows():
#         if int(i) == row['poi_cbg']:
            
#             for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
#                 if visitor_cbg == i:
#                     continue
#                 if visitor_cbg in algorithm_result[0]:
#                     movement_in_cbgs += num_visitors
#                 else:
#                     movement_out_cbgs +=num_visitors


#Calculate movement in and out for S
for i in algorithm_result[0]:
    adj = list(G.adj[i])
    for j in adj:
        if j in algorithm_result[0]:
            #over 2 to avoid double count in undirected graph
            movement_in_cbgs += G.adj[i][j]['weight']/2
        else:
            movement_out_cbgs += G.adj[i][j]['weight']
    

print("IN ", movement_in_cbgs, "OUT", movement_out_cbgs)

                
    
    
    

In [ ]:
algorithm_result = pagerank_nibble(G, core_cbg,0.6)
print(algorithm_result[0])
print('cluster population:', algorithm_result[1], '# of cbgs:', len(algorithm_result[0]))
movement_in_cbgs = 0
movement_out_cbgs = 0

pop = 0
for i in algorithm_result[0]:
    pop += cbg_population(i)
print("POP ", pop)


for i in algorithm_result[0]:
    adj = list(G.adj[i])
    for j in adj:
        if j in algorithm_result[0]:
            movement_in_cbgs += G.adj[i][j]['weight']/2
        else:
            movement_out_cbgs += G.adj[i][j]['weight']

print("IN ", movement_in_cbgs, "OUT", movement_out_cbgs, " Ratio ", movement_in_cbgs/(movement_in_cbgs+movement_out_cbgs))

                
    
    
    

In [19]:
print('starting algo')
algorithm_result = greedy_weight(G, core_cbg, min_cluster_pop)
print('algo done')
print('cluster population:', algorithm_result[1], '# of cbgs:', len(algorithm_result[0]))
print(algorithm_result[0])
movement_in_cbgs = 0
movement_out_cbgs = 0


for i in algorithm_result[0]:
    adj = list(G.adj[i])
    for j in adj:
        if j in algorithm_result[0]:
            movement_in_cbgs += G.adj[i][j]['weight']/2
        else:
            movement_out_cbgs += G.adj[i][j]['weight']
    

print("IN ", movement_in_cbgs, "OUT", movement_out_cbgs, " Ratio ", movement_in_cbgs/(movement_in_cbgs+movement_out_cbgs) )

                
    
    
    

starting algo
  Prev pop   59  New pop   137  Prev pop   137  New pop   179  Prev pop   179  New pop   207  Prev pop   207  New pop   284  Prev pop   284  New pop   328  Prev pop   328  New pop   346  Prev pop   346  New pop   393  Prev pop   393  New pop   450  Prev pop   450  New pop   552  Prev pop   552  New pop   592  Prev pop   592  New pop   681  Prev pop   681  New pop   714  Prev pop   714  New pop   790  Prev pop   790  New pop   854  Prev pop   854  New pop   911  Prev pop   911  New pop   950  Prev pop   950  New pop   1129  Prev pop   1129  New pop   1195  Prev pop   1195  New pop   1348  Prev pop   1348  New pop   1398  Prev pop   1398  New pop   1479  Prev pop   1479  New pop   1555  Prev pop   1555  New pop   1681  Prev pop   1681  New pop   1767  Prev pop   1767  New pop   1836  Prev pop   1836  New pop   1905  Prev pop   1905  New pop   1972  Prev pop   1972  New pop   2001  Prev pop   2001  New pop   2033  Prev pop   2033  New pop   2082  Prev pop   2082  New pop   2

In [ ]:
#For a cbg, see all the POI's and their visit data

#Put cbg number here
cbg = 240430103003

cbg_data = []


for _, row in df.iterrows():
    if cbg == row['poi_cbg']:
        data = {}
        data['location_name'] = row['location_name']
        data['median_dwell'] = row['median_dwell']
        visits = 0
        b_visits = 0
        for visitor_cbg, num_visitors in json.loads(row['visitor_daytime_cbgs']).items():
            visits += num_visitors
            if visitor_cbg == core_cbg:
                b_visits = num_visitors

        data['total_visits'] = visits
        data['core_visits'] = b_visits
        cbg_data.append(data)

#Sort data by total visits
sorted_cbg_data = sorted(cbg_data, key=lambda x: x['total_visits'], reverse=True)

#print
for data in sorted_cbg_data:
    print("Location ", data['location_name'], end="", flush=True)
    print(", Median dwell, ", data['median_dwell'], end="", flush=True)
    print(" ,Total Visits ", data['total_visits'], end="", flush=True)
    print(", core Visits ", data['core_visits'], end="", flush=True)
    print("")


Location  Long Meadow Shopping Center, Median dwell,  10.0 ,Total Visits  1525, core Visits  4
Location  McDonald's, Median dwell,  7.0 ,Total Visits  521, core Visits  0
Location  Dollar General, Median dwell,  20.0 ,Total Visits  479, core Visits  0
Location  CVS, Median dwell,  10.0 ,Total Visits  316, core Visits  0
Location  Rooster Moon Coffeehouse, Median dwell,  11.0 ,Total Visits  199, core Visits  4
Location  U.N.I. Urgent Care Center, Median dwell,  48.0 ,Total Visits  184, core Visits  0
Location  Free Range Cafe, Median dwell,  10.0 ,Total Visits  178, core Visits  0
Location  North Side Pizzeria, Median dwell,  40.0 ,Total Visits  157, core Visits  0
Location  Riehl's Bakery, Median dwell,  22.0 ,Total Visits  140, core Visits  0
Location  Fountain Head Country Club, Median dwell,  64.0 ,Total Visits  131, core Visits  0
Location  Spicher's Appliances, Median dwell,  15.0 ,Total Visits  129, core Visits  0
Location  Long Meadow Liquors, Median dwell,  8.0 ,Total Visits  1

In [ ]:
#TEST
print(G['401139400082']['401430034001'])

In [ ]:
#Plot graph representation
H = nx.subgraph(G, algorithm_result[0])
color_map = []
for node in H:
    frac = float(algorithm_result[0].index(node)) / len(algorithm_result[0])
    color_map.append('#%02X0000' % (int(frac * 255)))

pos_map = {}
for x in algorithm_result[0]:
    coords = cbg_geocode(x);
    pos_map[x] = (coords.latitude, coords.longitude)
    
print(pos_map)

nx.draw(H, node_color=color_map, pos=pos_map, with_labels=False)

KeyboardInterrupt: 

In [20]:
#MAP

#If you used the distance graph, reset G to the regular graph for mapping to see movement data
#G = gen_graph(df)

#use comments below if you want to copy/paste S
#algorithm_result = [[1],[1]]
#algorithm_result[0] =  

seed = cbg_geocode('240430006012')
center = (seed['latitude'], seed['longitude'])


count = 0

map = folium.Map(location=[center[0],center[1]], zoom_start=13)
total_in = 0
total_out = 0
zero_count = 0

for cbg in algorithm_result[0]:
    
   
    movement_in_S = 0
    movement_out_S = 0
    
    #get shape
    shape = gdf[gdf['GEOID10'] == cbg]
    
    if shape.empty:
        print("NO SHAPE",cbg)
        count +=1
    
    #calculate movement for a cbg
    edges = G.edges(cbg)
    for i in edges:
        
        if i[1] in algorithm_result[0]:

            movement_in_S += G[cbg][i[1]]['weight']/2
           
        else:
            movement_out_S += G[cbg][i[1]]['weight']
            
    
    #get ratio
    try:
        ratio = movement_in_S/(movement_in_S+movement_out_S)
    except ZeroDivisionError:
        zero_count +=1
        continue
        
    print("Cbg", cbg, " IN ", movement_in_S, " OUT ", movement_out_S, " RATIO ", ratio)
    total_in +=movement_in_S
    total_out+=movement_out_S
    
    
    #Shape color is determined by movement data
    # if ratio >= 0.9: #Purple
    #     layer = folium.GeoJson(shape,style_function=lambda feature: {
    #     'fillColor': '#800080','color': '#800080', 'fillOpacity':0.7}  ).add_to(map)
    if ratio >= 0.8: #Blue
        layer = folium.GeoJson(shape,style_function=lambda feature: {
        'fillColor': '#0000FF','color': '#0000FF', 'fillOpacity':0.7}  ).add_to(map)
    elif ratio >=0.6: #Green
        layer = folium.GeoJson(shape,style_function=lambda feature: {
        'fillColor': '#008000', 'color': '#008000','fillOpacity':0.7}  ).add_to(map)
    elif ratio >=0.4: #Yellow
        layer = folium.GeoJson(shape,style_function=lambda feature: {
        'fillColor': '#FFFF00', 'color': '#FFFF00','fillOpacity':0.7}  ).add_to(map)
    elif ratio >= 0.2: #Orange
        layer = folium.GeoJson(shape,style_function=lambda feature: {
        'fillColor': '#FFA500', 'color': '#FFA500','fillOpacity':0.7}  ).add_to(map)
    elif ratio >= 0: #Red
        layer = folium.GeoJson(shape,style_function=lambda feature: {
        'fillColor': '#FF0000', 'color': '#FF0000','fillOpacity':0.7}  ).add_to(map)


    layer.add_child(folium.Popup(cbg,sticky=True))
    
    #Show POI's on map, can be commented to only see cbgs
    for _, row in df.iterrows():
        if int(cbg) == row['poi_cbg']:
            poi_lat = poi_geocode(row['safegraph_place_id'])[1].iloc[0]
            poi_long = poi_geocode(row['safegraph_place_id'])[2].iloc[0]
            location = (poi_lat,poi_long)
            name = row['location_name']+' CBG: '+ str(row['poi_cbg'])
            pop = folium.Popup(name,sticky=True)
            folium.CircleMarker([poi_lat, poi_long],radius = 5, color="#3186cc", fill=True,fill_color="#3186cc",popup=pop,sticky=True).add_to(map)
            
    
print("exited loop", "Zero_division ", zero_count)
print("COUNT",count)
print("IN ",total_in, "Out ",total_out, "Ratio ",total_in/(total_in+total_out))
#map
print('saving')
map.save("output.html")
print('saved')


Cbg 240430006012  IN  52333.0  OUT  26670  RATIO  0.6624178828652076
Cbg 240430006011  IN  225557.0  OUT  162554  RATIO  0.581166212758721
Cbg 240430112013  IN  361799.0  OUT  301192  RATIO  0.5457072569612559
Cbg 240430009003  IN  569767.0  OUT  991874  RATIO  0.3648514607390559
Cbg 240430109001  IN  243053.0  OUT  199698  RATIO  0.5489609283773498
Cbg 240430104004  IN  319052.0  OUT  323736  RATIO  0.49635649700990064
Cbg 240430010011  IN  235289.0  OUT  291396  RATIO  0.44673571489600045
Cbg 240430105003  IN  116843.0  OUT  78834  RATIO  0.5971217874354166
Cbg 240430010023  IN  177398.0  OUT  197572  RATIO  0.47309918126783473
Cbg 240430006021  IN  107535.0  OUT  58978  RATIO  0.6458054326088654
Cbg 240430006022  IN  162229.0  OUT  134170  RATIO  0.5473331556449246
Cbg 240430112021  IN  156910.0  OUT  92072  RATIO  0.6302061996449543
Cbg 240430103001  IN  195101.0  OUT  228016  RATIO  0.46110413904428327
Cbg 240430103003  IN  150529.0  OUT  101942  RATIO  0.5962229325348258
Cbg 2404

In [ ]:
#UNFINISHED CODE
#The cbgs in S sometimes surround another cbg that is not in S. The goal is to include this cbg in S. This code will include the cbg, but also include cbgs that are not surrounded.
#This code works by adding cbgs outside S that share a border with at least 3 cbgs in S.

#Surrounded CBGs
cbg_n = {}
#algo_copy is the new S
algo_copy = algorithm_result[0].copy()
for cbg in algorithm_result[0]:
    try:
        row = gdf[gdf['GEOID10'] == cbg].iloc[0]
    except IndexError:
        continue
        
    #find all neibouring cbgs 
    current_neighbours = gdf[gdf.geometry.touches(row['geometry'])]
    print("N ",current_neighbours['GEOID10'])
    for i in current_neighbours['GEOID10']:
        #count the neihbours in S
        if i in algorithm_result[0]:
            continue

        if i not in cbg_n:
            cbg_n[i] = 1
        else:
            cbg_n[i] +=1

for key in cbg_n:
    #if more than 3 borders are shared with S
    if cbg_n[key] >=3 :
        print('found')
        shape = gdf[gdf['GEOID10'] == key]
        layer = folium.GeoJson(shape,style_function=lambda feature: {
        'fillColor': '#000000', 'color': '#000000','fillOpacity':0.7}  ).add_to(map)
        layer.add_child(folium.Popup(key))
        algo_copy.append(key)
print('saving')
map.save("pagerank_10000.html")
        
    




N  1559    240430006022
1576    240430006011
1577    240430005002
1580    240430005003
1581    240430007001
Name: GEOID10, dtype: object
N  1559    240430006022
1580    240430005003
1586    240430112012
1587    240430006012
1607    240430112013
Name: GEOID10, dtype: object
N  1551    240430112011
1557    240430112022
1563    240430111001
1576    240430006011
1580    240430005003
1583    240430001001
1586    240430112012
1598    240430112021
1613    240430112014
Name: GEOID10, dtype: object
N  1524    240430105004
1530    240430008001
1531    240430008002
1533    240430009002
1535    240430009001
1536    240430010014
1537    240430010011
1569    240430010012
1591    240430004003
1592    240430010021
1601    240430008003
1605    240430010023
Name: GEOID10, dtype: object
N  1537    240430010011
1538    240430010013
1544    240430110003
1558    240430006021
1560    240430110001
1561    240430111004
1564    240430109002
1606    240430109004
Name: GEOID10, dtype: object
N  1524    2404301050

In [ ]:
print(gdf.columns)

In [ ]:
#Plot every cbg (in Pen and MD here)
gdf1 = gpd.read_file('tl_2010_24_bg10/tl_2010_24_bg10.shp')

gdf2 = gpd.read_file('tl_2010_42_bg10/tl_2010_42_bg10.shp')


gdf = gpd.GeoDataFrame(pd.concat([gdf1, gdf2], ignore_index=True))
seed = cbg_geocode('240430006012')
center = (seed['latitude'], seed['longitude'])

map = folium.Map(location=[center[0],center[1]], zoom_start=13)


for index, row in gdf.iterrows():
    #Get cbg name
    cbg_name = row['GEOID10']
    
    #popup to show name of cbg
    popup = folium.Popup(str(cbg_name), sticky=True, parse_html=True)
    
   
    folium.GeoJson(row['geometry'], popup=popup).add_to(map)

# Show the map
map.save('all_cbgs.html')


In [ ]:
#Get the ratio for multiple populations
R = []
G_r = gen_graph(df)
for k in range(1,21):
    print('starting algo ', k)
    algorithm_result = greedy_weight(G, core_cbg, k*1000)
    print('algo done')
    print('cluster population:', algorithm_result[1], '# of cbgs:', len(algorithm_result[0]))
    print(algorithm_result[0])
    movement_in_cbgs = 0
    movement_out_cbgs = 0
   

    for i in algorithm_result[0]:
        adj = list(G_r.adj[i])
        for j in adj:
            if j in algorithm_result[0]:
                movement_in_cbgs += G_r.adj[i][j]['weight']/2
            else:
                movement_out_cbgs += G_r.adj[i][j]['weight']
        

    print("POP ", i, " IN ", movement_in_cbgs, "OUT", movement_out_cbgs, " Ratio ", movement_in_cbgs/(movement_in_cbgs+movement_out_cbgs) )
    R.append(movement_in_cbgs/(movement_in_cbgs+movement_out_cbgs))
    
    
    #MAP the population
    seed = cbg_geocode('240430006012')
    center = (seed['latitude'], seed['longitude'])



    
    map = folium.Map(location=[center[0],center[1]], zoom_start=13)
    total_in = 0
    total_out = 0
    zero_count = 0
    for cbg in algorithm_result[0]:
        
        
        movement_in_S = 0
        movement_out_S = 0
        
        shape = gdf[gdf['GEOID10'] == cbg]

        edges = G_r.edges(cbg)
        for i in edges:
            
            if i[1] in algorithm_result[0]:

                movement_in_S += G_r[cbg][i[1]]['weight']/2
            
            else:
                movement_out_S += G_r[cbg][i[1]]['weight']
                
        
        
        try:
            ratio = movement_in_S/(movement_in_S+movement_out_S)
        except ZeroDivisionError:
            zero_count +=1
            continue
            
        print("Cbg", cbg, " IN ", movement_in_S, " OUT ", movement_out_S, " RATIO ", ratio)
        total_in +=movement_in_S
        total_out+=movement_out_S
        

        if ratio >= 0.8:
            layer = folium.GeoJson(shape,style_function=lambda feature: {
            'fillColor': '#0000FF','color': '#0000FF', 'fillOpacity':0.7}  ).add_to(map)
        elif ratio >=0.6:
            layer = folium.GeoJson(shape,style_function=lambda feature: {
            'fillColor': '#008000', 'color': '#008000','fillOpacity':0.7}  ).add_to(map)
        elif ratio >=0.4:
            layer = folium.GeoJson(shape,style_function=lambda feature: {
            'fillColor': '#FFFF00', 'color': '#FFFF00','fillOpacity':0.7}  ).add_to(map)
        elif ratio >= 0.2:
            layer = folium.GeoJson(shape,style_function=lambda feature: {
            'fillColor': '#FFA500', 'color': '#FFA500','fillOpacity':0.7}  ).add_to(map)
        elif ratio >= 0:
            layer = folium.GeoJson(shape,style_function=lambda feature: {
            'fillColor': '#FF0000', 'color': '#FF0000','fillOpacity':0.7}  ).add_to(map)


            





        
    
        
        layer.add_child(folium.Popup(cbg))
        # for _, row in df.iterrows():
        #     if int(cbg) == row['poi_cbg']:
        #         poi_lat = poi_geocode(row['safegraph_place_id'])[1].iloc[0]
        #         poi_long = poi_geocode(row['safegraph_place_id'])[2].iloc[0]
        #         location = (poi_lat,poi_long)
        #         folium.CircleMarker([poi_lat, poi_long],radius = 5, color="#3186cc", fill=True,fill_color="#3186cc",popup=row['location_name']).add_to(map)
                
        
    print("exited loop", "Zero_division ", zero_count)

    print("IN ",total_in, "Out ",total_out, "Ratio ",total_in/(total_in+total_out))
    #map
    print('saving')
    name = "greedy_d_" + str(k) + ".html"
    map.save(name)
    print('saved')


In [ ]:
#Plot Ratio vs Population
import matplotlib.pyplot as plt


x = [1, 2, 3, 4, 5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
#R is found in the above cell
y = R

#Plot
plt.plot(x, y, marker='o', linestyle='-', color='b')
plt.xticks(range(1, 21))
#Labels
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Ratio vs Population')
plt.legend()


plt.grid(True)
plt.show()

    



In [ ]:
#Map final cbg states as animation

movement_in_S = 0
movement_out_S = 0

seed = cbg_geocode('240430006012')

center = (seed['latitude'], seed['longitude'])
map = folium.Map(location=[center[0],center[1]], zoom_start=13)

features = []
for i, cbg in enumerate(algorithm_result[0]):
    
    movement_in_S = 0
    movement_out_S = 0
    shape = gdf[gdf['GEOID10'] == cbg]

    edges = G.edges(cbg)
    for j in edges:
        
        if j[1] in algorithm_result[0]:

            movement_in_S += G[cbg][j[1]]['weight']/2
           
        else:
            movement_out_S += G[cbg][j[1]]['weight']
            
    
    ratio = movement_in_S/(movement_in_S+movement_out_S)
    # try:
    #     ratio = movement_in_S/(movement_in_S+movement_out_S)
    # except ZeroDivisionError:
    #     zero_count +=1
    #     continue

    if ratio >= 0.8:
        fillcolor = '#0000FF'
        color = '#0000FF'
       
    elif ratio >=0.6:
        fillcolor = '#008000'
        color = '#008000'
       
    elif ratio >=0.4:
        fillcolor = '#FFFF00'
        color = '#FFFF00'
       
    elif ratio >= 0.2:
        fillcolor = '#FFA500'
        color = '#FFA500'
        
    elif ratio >= 0:
        fillcolor = '#FF0000'
        color = '#FF0000'
       


    
    # Check if shape is not empty
    if not shape.empty:
        # Convert to EPSG:4326
        shape = shape.to_crs("EPSG:4326")
        # Convert to GeoJSON
        geojson = json.loads(shape.to_json())
        feature = geojson['features'][0]
        # Add a times property to the feature
        feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'D')).isoformat()]
        feature['properties']['style'] = {'fillColor': fillcolor, 'color': color, 'fillOpacity': 0.7}
        features.append(feature)

plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT6H',
    add_last_point=True
).add_to(map)

map


In [ ]:
#FUNCTIONS
#get the color of a shape depending on its ratio
def get_color(G,lst,cbg):
    movement_in_S = 0
    movement_out_S = 0
    edges = G.edges(cbg)
    for j in edges:
        
        if j[1] in lst:

            movement_in_S += G[cbg][j[1]]['weight']/2
           
        else:
            movement_out_S += G[cbg][j[1]]['weight']
            
    
    ratio = movement_in_S/(movement_in_S+movement_out_S)
    
   

    if ratio >= 0.8:
        #Blue
        color = '#0000FF'   
       
    elif ratio >=0.6:
        #Green
        color = '#008000'
       
    elif ratio >=0.4:
        #Yellow
        color = '#FFFF00'
       
    elif ratio >= 0.2:
        #Orange
        color = '#FFA500'
        
    elif ratio >= 0:
        #Red
        color = '#FF0000'
    
    return color


#Create a feature for mapping
def create_feature(shape, color, timestamp, features):
    if not shape.empty:
        shape = shape.to_crs("EPSG:4326")
        # Convert to GeoJSON
        geojson = json.loads(shape.to_json())
        feature = geojson['features'][0]
        # Add a times property to the feature
        feature['properties']['times'] = [(pd.Timestamp('today') + pd.Timedelta(i, 'D')).isoformat()]
        feature['properties']['style'] = {'fillColor': color, 'color': color, 'fillOpacity': 0.7}
        features.append(feature)
    

In [ ]:
#Animated map where S is all cbgs showing on the map. When a new cbg is added to S, all shapes on the map update their colors to reflect movement data
#Folium cannot change the color of a feature once it has been added, instead the new color is printed over.
seed = cbg_geocode('240430006012')
center = (seed['latitude'], seed['longitude'])
map = folium.Map(location=[center[0], center[1]], zoom_start=13)

#lst is the cbgs currently showing on the map
lst = []
features = []

for i, cbg in enumerate(algorithm_result[0]):
    print(cbg)
    lst.append(cbg)
    timestamp = (pd.Timestamp('today') + pd.Timedelta(i, 'D')).isoformat()
    #Add new shape
    shape = gdf[gdf['GEOID10'] == cbg]
    color = get_color(G, lst, cbg) 
    create_feature(shape, color, timestamp, features)


    #Update colors for old shapes
    for j in lst[:-1]:
        shape = gdf[gdf['GEOID10'] == j]
        color = get_color(G, lst, j)
        create_feature(shape, color, timestamp, features)
        

    




plugins.TimestampedGeoJson(
    {'type': 'FeatureCollection', 'features': features},
    period='PT6H',
    
    add_last_point=True
).add_to(map)

map.save('animated.html')


240430006012
240430006011
240430112013
240430009003
240430109001
240430104004
240430010011
240430105003
240430010023
240430006021
240430006022
240430112021
240430103001
240430103003
240430103002
240430002002
240430104002
240430113013
240430112014
240430106001
240430104003
240430003012
240430108011
240430108021
240430108012
240430004003
240430111003
240430005003
240430111001
240430108022
240430007001
240430010022
240430005001
240430114001
240430114002
240430115001
240430116001
240430010013
240430105001
240430111004
240430112011
240430003011
240430111002
240430112012
240430105004
240430109004
240430004001
240430004002
240430003022
240430009001
240430009002
240430002001
240430005002
240430007002
240430107002
420550125012
420550109001
420550117002
420550118001
420550118002
420550106001
420550117004
420550118003
420550117003
420550119003
420550124002
420550125011
420550108003
420550122001
420550123001
420550121001
420550125022
420550119004
240217510033
240217510031
240430102004
420550107001